### variable small & cow many
데이터에서 변수를 최대한 제거하고 학습에 사용할 cow의 양을 늘리는 방법으로 전처리 진행

| 데이터 | unit   | process | variable           |
|--------|--------|---------|--------------------|
| 체중   | day    | merge   | 체중               |
| 급유   | second | merge   | 착유 횟수 , 횟수량 |
| 번식   | day    | for     | 임신기간           |
| 환경   | day    | merge   | CO2, 온도,습도     |
| 급이   | day    | merge   | 급이량             |

사용된 변수 및 처리 방법

### dataload 

In [1]:
import pandas as pd

weight_data = pd.read_excel('./raw_data/낙농_ICT데이터_체중정보.xlsx', header=1)
milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')

breeding_data = pd.read_excel('./raw_data/낙농_개체별_번식이력.xlsx' , header=1)

env1_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경관리기정보.xlsx' , header=1)
env2_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경제어기정보.xlsx' , header=1)

gen_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_일반급이기정보.xlsx' , header=1)
ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
tmr_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_TMR급이기정보.xlsx' , header=1)
milk_cls_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_착유군사급이기정보.xlsx' , header=1)


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-1-88f63fd9ca52>:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no defaul

#### 하나의 개체에 대해서 dataframe으로 만들어서 저장 

In [2]:
# 개체명 , 농가명 별로 dataframe을 만들어서 저장
cow_id = weight_data['개체번호'].unique()[0]
cow_data = weight_data[weight_data['개체번호'] == cow_id]
farm_id = cow_data['농가명'].unique()[0]

cow_data = cow_data.sort_values(by='측정일시')
cow_data

,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,체중(kg)
234,235,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-01-21 00:00,639.0
233,234,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-01-22 00:00,643.0
232,233,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-01-23 00:00,646.0
231,232,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-01-24 00:00,653.0
230,231,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-01-25 00:00,655.0
...,...,...,...,...,...,...,...,...,...
4,5,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-07 00:00,750.0
3,4,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-08 00:00,750.0
2,3,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-09 00:00,749.0
1,2,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-10 00:00,751.0


In [3]:
# to_csv(f'{farm_id}_{cow_id}.csv')

In [4]:
# time_idx 생성 및 체중 dataframe 생성
start_date = '2009-01-01'
end_date = '2023-08-31'

dt = pd.date_range(start=start_date, end=end_date, freq='D')
cow_df = pd.DataFrame(index = dt , columns = ['측정일시'])
cow_df

,측정일시
2009-01-01,NaN
2009-01-02,NaN
2009-01-03,NaN
2009-01-04,NaN
2009-01-05,NaN
...,...
2023-08-27,NaN
2023-08-28,NaN
2023-08-29,NaN
2023-08-30,NaN


### weight data 가져오기
 

In [5]:
weight_st = cow_data[cow_data['측정일시'].notnull()]['측정일시'].to_list()
cow_df['측정일시'] = cow_df.index.isin(weight_st).astype(int)
cow_df[cow_df['측정일시'] > 0]

,측정일시
2020-01-21,1
2020-01-22,1
2020-01-23,1
2020-01-24,1
2020-01-25,1
...,...
2020-09-07,1
2020-09-08,1
2020-09-09,1
2020-09-10,1


### milk data 전처리 

In [6]:
milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')
milk_data.info()

<ipython-input-6-c5f7b86ecf56>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822740 entries, 0 to 822739
Data columns (total 23 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  822740 non-null  int64  
 1   No          822740 non-null  int64  
 2   지역          51542 non-null   float64
 3   규모          822740 non-null  object 
 4   농가명         822740 non-null  int64  
 5   개체번호        822740 non-null  int64  
 6   제조사아이디      822740 non-null  object 
 7   장비번호        822740 non-null  int64  
 8   측정일시        822740 non-null  object 
 9   착유량(L)      822740 non-null  int64  
 10  착유시작일시      812283 non-null  object 
 11  착유종료일시      812283 non-null  object 
 12  착유회차        745440 non-null  float64
 13  전도도         812283 non-null  float64
 14  체세포         100046 non-null  float64
 15  혈액흐름        734983 non-null  object 
 16  색상          24457 non-null   float64
 17  온도(℃)       734982 non-null  float64
 18  유지방         767865 non-null  float64
 19  유단

In [7]:
milk_data = milk_data[milk_data['개체번호'] == cow_id]
milk_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),...,전도도,체세포,혈액흐름,색상,온도(℃),유지방,유단백,반추시간,공기흐름,산차
7512,7512,7513,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-05-31 00:00,39,...,56.50,20.0,NaN,NaN,NaN,3.84,2.88,552.0,NaN,6.0
7513,7513,7514,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-05-30 00:00,39,...,58.25,32.0,NaN,NaN,NaN,3.89,3.04,596.0,NaN,6.0
7514,7514,7515,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-05-29 00:00,37,...,57.25,6.0,NaN,NaN,NaN,4.08,3.00,527.0,NaN,6.0
7515,7515,7516,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-05-28 00:00,38,...,58.50,42.0,NaN,NaN,NaN,4.60,2.89,594.0,NaN,6.0
7516,7516,7517,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-05-27 00:00,40,...,51.00,28.0,NaN,NaN,NaN,4.74,2.86,425.0,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819635,819635,6561,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-02-05 00:00,36,...,56.50,46.0,NaN,NaN,NaN,3.97,3.07,499.0,NaN,6.0
819636,819636,6562,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-02-04 00:00,43,...,58.25,156.0,NaN,NaN,NaN,3.86,3.05,517.0,NaN,6.0
819637,819637,6563,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-02-03 00:00,35,...,58.00,28.0,NaN,NaN,NaN,3.84,3.00,505.0,NaN,6.0
819638,819638,6564,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,2020-02-02 00:00,40,...,57.25,18.0,NaN,NaN,NaN,4.02,2.94,583.0,NaN,6.0


In [8]:
# 필요한 데이터만 가져옴
milk_data = milk_data[['개체번호' , '착유량(L)' , '측정일시']]
milk_data

,개체번호,착유량(L),측정일시
7512,20120112010022,39,2020-05-31 00:00
7513,20120112010022,39,2020-05-30 00:00
7514,20120112010022,37,2020-05-29 00:00
7515,20120112010022,38,2020-05-28 00:00
7516,20120112010022,40,2020-05-27 00:00
...,...,...,...
819635,20120112010022,36,2020-02-05 00:00
819636,20120112010022,43,2020-02-04 00:00
819637,20120112010022,35,2020-02-03 00:00
819638,20120112010022,40,2020-02-02 00:00


In [9]:
# 시간 단위로 전처리
milk_data['측정일시'] = pd.to_datetime(milk_data['측정일시'])
milk_data['측정일시'] = milk_data['측정일시'].dt.floor('H')  # 'H' stands for hour
milk_data = milk_data.sort_values(by='측정일시')
milk_data

<ipython-input-9-023a1f1bc4f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milk_data['측정일시'] = pd.to_datetime(milk_data['측정일시'])
<ipython-input-9-023a1f1bc4f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milk_data['측정일시'] = milk_data['측정일시'].dt.floor('H')  # 'H' stands for hour


,개체번호,착유량(L),측정일시
446928,20120112010022,24,2020-01-20
19891,20120112010022,24,2020-01-20
19890,20120112010022,28,2020-01-21
446927,20120112010022,28,2020-01-21
446926,20120112010022,37,2020-01-22
...,...,...,...
384469,20120112010022,20,2020-09-07
384468,20120112010022,22,2020-09-08
384467,20120112010022,22,2020-09-09
384466,20120112010022,21,2020-09-10


In [10]:
# groupby 연산을 이용해서 일일 단위로 데이터를 집계
daily_counts = milk_data.groupby([milk_data['측정일시'].dt.date, '개체번호']).size().reset_index(name='일일 착유 횟수')
daily_milk = milk_data.groupby([milk_data['측정일시'].dt.date, '개체번호'])['착유량(L)'].sum().reset_index()

milk_agg_df = pd.merge(daily_counts, daily_milk, on=['측정일시', '개체번호'])
milk_agg_df.set_index('측정일시' , inplace=True)
milk_agg_df

,개체번호,일일 착유 횟수,착유량(L)
측정일시,,,
2020-01-20,20120112010022,2,48
2020-01-21,20120112010022,2,56
2020-01-22,20120112010022,2,74
2020-01-23,20120112010022,2,86
2020-01-24,20120112010022,2,86
...,...,...,...
2020-09-07,20120112010022,1,20
2020-09-08,20120112010022,1,22
2020-09-09,20120112010022,1,22


In [11]:
# 제대로 merge 되었는지 확인
merge_df = pd.merge(cow_df, milk_agg_df[['일일 착유 횟수' , '착유량(L)']], how='outer' , left_index=True, right_index=True)
merge_df

,측정일시,일일 착유 횟수,착유량(L)
2009-01-01,0,NaN,NaN
2009-01-02,0,NaN,NaN
2009-01-03,0,NaN,NaN
2009-01-04,0,NaN,NaN
2009-01-05,0,NaN,NaN
...,...,...,...
2023-08-27,0,NaN,NaN
2023-08-28,0,NaN,NaN
2023-08-29,0,NaN,NaN
2023-08-30,0,NaN,NaN


In [12]:
merge_df[merge_df['측정일시'] > 0]

,측정일시,일일 착유 횟수,착유량(L)
2020-01-21,1,2.0,56.0
2020-01-22,1,2.0,74.0
2020-01-23,1,2.0,86.0
2020-01-24,1,2.0,86.0
2020-01-25,1,2.0,88.0
...,...,...,...
2020-09-07,1,1.0,20.0
2020-09-08,1,1.0,22.0
2020-09-09,1,1.0,22.0
2020-09-10,1,1.0,21.0


In [13]:
merge_df[merge_df['일일 착유 횟수'].notnull()]

,측정일시,일일 착유 횟수,착유량(L)
2020-01-20,0,2.0,48.0
2020-01-21,1,2.0,56.0
2020-01-22,1,2.0,74.0
2020-01-23,1,2.0,86.0
2020-01-24,1,2.0,86.0
...,...,...,...
2020-09-07,1,1.0,20.0
2020-09-08,1,1.0,22.0
2020-09-09,1,1.0,22.0
2020-09-10,1,1.0,21.0


### 번식데이터 

In [14]:
breeding_data = pd.read_excel('./raw_data/낙농_개체별_번식이력.xlsx' , header=1)
breeding_data

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,No,지역,규모,농가명,개체번호,산차,수정차수,수정일,수정유형,정액코드,건유일,분만일,임신기간(일),종부횟수,공태일수,분만간격(일),초산일령
0,1,3.100000e+09,300~400,20276,20141023010133,5,1,2022-01-22,인공수정,7HO13930,NaN,NaN,NaN,1,NaN,NaN,NaN
1,2,3.100000e+09,300~400,20276,20141023010133,4,3,2021-03-30,인공수정,7HO12796,2021-10-31,2021-12-30,275.0,3,583.0,858.0,NaN
2,3,3.100000e+09,300~400,20276,20141023010133,4,2,2020-04-29,인공수정,7HO12206,NaN,NaN,NaN,2,NaN,NaN,NaN
3,4,3.100000e+09,300~400,20276,20141023010133,4,1,2019-10-20,인공수정,7HO11316,NaN,NaN,NaN,1,NaN,NaN,NaN
4,5,3.100000e+09,300~400,20276,20141023010133,3,2,2018-11-19,인공수정,250HO11837,2019-06-25,2019-08-25,279.0,2,87.0,366.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31110,31111,NaN,500~1000,20249,20210802020406,0,1,2022-09-30,인공수정,200HO10845,2022-10-22,2022-11-23,54.0,1,NaN,NaN,479.0
31111,31112,NaN,500~1000,20249,20210809020404,0,1,2022-09-27,인공수정,200HO10845,2022-10-22,2022-11-23,57.0,1,NaN,NaN,472.0
31112,31113,NaN,500~1000,20249,20210815020403,0,1,2022-11-11,인공수정,200HO10845,2022-10-22,2022-11-23,12.0,1,NaN,NaN,466.0
31113,31114,NaN,500~1000,20249,20210901020402,0,1,2022-11-11,인공수정,203HO01468,2022-10-22,2022-11-23,12.0,1,NaN,NaN,449.0


In [15]:
breeding_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31115 entries, 0 to 31114
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       31115 non-null  int64  
 1   지역       9151 non-null   float64
 2   규모       31115 non-null  object 
 3   농가명      31115 non-null  int64  
 4   개체번호     31115 non-null  int64  
 5   산차       31115 non-null  int64  
 6   수정차수     31115 non-null  int64  
 7   수정일      31115 non-null  object 
 8   수정유형     31115 non-null  object 
 9   정액코드     28175 non-null  object 
 10  건유일      7077 non-null   object 
 11  분만일      11662 non-null  object 
 12  임신기간(일)  11662 non-null  float64
 13  종부횟수     31115 non-null  int64  
 14  공태일수     6208 non-null   float64
 15  분만간격(일)  6208 non-null   float64
 16  초산일령     4778 non-null   float64
dtypes: float64(5), int64(6), object(6)
memory usage: 4.0+ MB


In [16]:
# 필요한 데이터만 가져옴
breeding_data = breeding_data[['개체번호' , '산차' , '수정일' , '수정차수' , '분만일']]

In [17]:
# 분만일이 없는 경우 임신기간에 대해서 계산이 불가하므로 제외함
breeding_data = breeding_data[breeding_data['개체번호'] == cow_id]
breeding_data = breeding_data[breeding_data['분만일'].notnull()]
breeding_data

,개체번호,산차,수정일,수정차수,분만일
19400,20120112010022,5,2018-12-16,1,2019-09-26
19401,20120112010022,4,2017-12-17,1,2018-09-17
19402,20120112010022,3,2016-10-06,1,2017-07-13
19403,20120112010022,2,2015-09-05,2,2016-06-11
19405,20120112010022,1,2014-05-10,1,2015-02-16
19406,20120112010022,0,2013-04-05,1,2014-01-13


In [18]:
breeding_data['수정일'] = pd.to_datetime(breeding_data['수정일'])
breeding_data['분만일'] = pd.to_datetime(breeding_data['분만일'])
breeding_data

,개체번호,산차,수정일,수정차수,분만일
19400,20120112010022,5,2018-12-16,1,2019-09-26
19401,20120112010022,4,2017-12-17,1,2018-09-17
19402,20120112010022,3,2016-10-06,1,2017-07-13
19403,20120112010022,2,2015-09-05,2,2016-06-11
19405,20120112010022,1,2014-05-10,1,2015-02-16
19406,20120112010022,0,2013-04-05,1,2014-01-13


In [19]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5356 entries, 2009-01-01 to 2023-08-31
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   측정일시      5356 non-null   int64  
 1   일일 착유 횟수  236 non-null    float64
 2   착유량(L)    236 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 167.4 KB


In [20]:
# start , end 기간의 형식이므로 date_range를 이용해서 time stamp을 이용해서 기간을 구하고 기간을 넣음 
for index, row in breeding_data.iterrows():
    start, end = row['수정일'], row['분만일']
    print(start , end)
    pregnancy_dt = pd.date_range(start, end , freq = 'D')
    merge_df.loc[pregnancy_dt, '임신기간'] = 1

merge_df['임신기간'].fillna(0, inplace=True)

2018-12-16 00:00:00 2019-09-26 00:00:00
2017-12-17 00:00:00 2018-09-17 00:00:00
2016-10-06 00:00:00 2017-07-13 00:00:00
2015-09-05 00:00:00 2016-06-11 00:00:00
2014-05-10 00:00:00 2015-02-16 00:00:00
2013-04-05 00:00:00 2014-01-13 00:00:00


In [21]:
for x in merge_df[merge_df['임신기간'] > 0].index:
    print(x)

2013-04-05 00:00:00
2013-04-06 00:00:00
2013-04-07 00:00:00
2013-04-08 00:00:00
2013-04-09 00:00:00
2013-04-10 00:00:00
2013-04-11 00:00:00
2013-04-12 00:00:00
2013-04-13 00:00:00
2013-04-14 00:00:00
2013-04-15 00:00:00
2013-04-16 00:00:00
2013-04-17 00:00:00
2013-04-18 00:00:00
2013-04-19 00:00:00
2013-04-20 00:00:00
2013-04-21 00:00:00
2013-04-22 00:00:00
2013-04-23 00:00:00
2013-04-24 00:00:00
2013-04-25 00:00:00
2013-04-26 00:00:00
2013-04-27 00:00:00
2013-04-28 00:00:00
2013-04-29 00:00:00
2013-04-30 00:00:00
2013-05-01 00:00:00
2013-05-02 00:00:00
2013-05-03 00:00:00
2013-05-04 00:00:00
2013-05-05 00:00:00
2013-05-06 00:00:00
2013-05-07 00:00:00
2013-05-08 00:00:00
2013-05-09 00:00:00
2013-05-10 00:00:00
2013-05-11 00:00:00
2013-05-12 00:00:00
2013-05-13 00:00:00
2013-05-14 00:00:00
2013-05-15 00:00:00
2013-05-16 00:00:00
2013-05-17 00:00:00
2013-05-18 00:00:00
2013-05-19 00:00:00
2013-05-20 00:00:00
2013-05-21 00:00:00
2013-05-22 00:00:00
2013-05-23 00:00:00
2013-05-24 00:00:00


In [22]:
merge_df

,측정일시,일일 착유 횟수,착유량(L),임신기간
2009-01-01,0,NaN,NaN,0.0
2009-01-02,0,NaN,NaN,0.0
2009-01-03,0,NaN,NaN,0.0
2009-01-04,0,NaN,NaN,0.0
2009-01-05,0,NaN,NaN,0.0
...,...,...,...,...
2023-08-27,0,NaN,NaN,0.0
2023-08-28,0,NaN,NaN,0.0
2023-08-29,0,NaN,NaN,0.0
2023-08-30,0,NaN,NaN,0.0


### 환경 정보 

In [23]:
env1_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경관리기정보.xlsx' , header=1)
env2_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경제어기정보.xlsx' , header=1)


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [24]:
env1_data = env1_data[env1_data['농가명'] == farm_id]
env1_data = env1_data.sort_values(by='측정일시')
env1_data.rename(columns={'측정값(℃ · % · ppm)': '측정값'}, inplace=True)
env1_data

,No,지역,규모,농가명,장비종류,제조사아이디,장비번호,측정일시,측정값
8403,8404,4.100000e+09,200~300,20278,온도,KILSOFT,1,2019-12-06,28.00
7970,7971,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-27,610.00
8185,8186,4.100000e+09,200~300,20278,습도,KILSOFT,1,2020-07-27,73.00
8402,8403,4.100000e+09,200~300,20278,온도,KILSOFT,1,2020-07-27,29.00
8184,8185,4.100000e+09,200~300,20278,습도,KILSOFT,1,2020-07-28,80.00
...,...,...,...,...,...,...,...,...,...
8188,8189,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-29,7.75
8187,8188,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-30,5.92
7972,7973,4.100000e+09,200~300,20278,습도,thekpmcorp,1,2022-12-30,73.00
8186,8187,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-31,75.96


In [25]:
env1_data[env1_data['장비종류'] == 'CO2']

,No,지역,규모,농가명,장비종류,제조사아이디,장비번호,측정일시,측정값
7970,7971,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-27,610.0
7969,7970,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-28,607.0
7968,7969,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-29,601.0
7967,7968,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-30,603.0
7966,7967,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-31,610.0
...,...,...,...,...,...,...,...,...,...
7882,7883,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-12-14,570.0
7881,7882,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-12-15,560.0
7880,7881,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-12-16,560.0
7879,7880,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2021-01-29,571.0


In [26]:
CO2_data = env1_data[env1_data['장비종류'] == 'CO2'][['측정일시' , '측정값']]
CO2_data.set_index('측정일시' , inplace = True)
CO2_data

,측정값
측정일시,
2020-07-27,610.0
2020-07-28,607.0
2020-07-29,601.0
2020-07-30,603.0
2020-07-31,610.0
...,...
2020-12-14,570.0
2020-12-15,560.0
2020-12-16,560.0


In [27]:
merge_df = pd.merge(merge_df, CO2_data, left_index=True, right_index=True, how='outer')
merge_df.rename(columns={'측정값': 'CO2 측정값'}, inplace=True)
merge_df

,측정일시,일일 착유 횟수,착유량(L),임신기간,CO2 측정값
2009-01-01 00:00:00,0.0,NaN,NaN,0.0,NaN
2009-01-02 00:00:00,0.0,NaN,NaN,0.0,NaN
2009-01-03 00:00:00,0.0,NaN,NaN,0.0,NaN
2009-01-04 00:00:00,0.0,NaN,NaN,0.0,NaN
2009-01-05 00:00:00,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...
2020-12-14,NaN,NaN,NaN,NaN,570.0
2020-12-15,NaN,NaN,NaN,NaN,560.0
2020-12-16,NaN,NaN,NaN,NaN,560.0
2021-01-29,NaN,NaN,NaN,NaN,571.0


In [28]:
merge_df[merge_df['CO2 측정값'] > 0]

,측정일시,일일 착유 횟수,착유량(L),임신기간,CO2 측정값
2020-07-27,NaN,NaN,NaN,NaN,610.0
2020-07-28,NaN,NaN,NaN,NaN,607.0
2020-07-29,NaN,NaN,NaN,NaN,601.0
2020-07-30,NaN,NaN,NaN,NaN,603.0
2020-07-31,NaN,NaN,NaN,NaN,610.0
...,...,...,...,...,...
2020-12-14,NaN,NaN,NaN,NaN,570.0
2020-12-15,NaN,NaN,NaN,NaN,560.0
2020-12-16,NaN,NaN,NaN,NaN,560.0
2021-01-29,NaN,NaN,NaN,NaN,571.0


나중에 CO2, 온도 , 습도 도 같이하는 코드 만들기 

### 급이 데이터 

In [29]:
gen_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_일반급이기정보.xlsx' , header=1)
ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
tmr_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_TMR급이기정보.xlsx' , header=1)
milk_cls_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_착유군사급이기정보.xlsx' , header=1)

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-29-c07edaf0b2be>:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### 중복되는 데이터 확인 

In [30]:
weight_data['농가명'].unique()

array([20278, 20264, 20279, 21133])

In [31]:
set(gen_feed_data['농가명'].unique()) & set(weight_data['농가명'].unique())

set()

In [32]:
set(ict_cls_feed_data['농가명'].unique()) & set(weight_data['농가명'].unique())

{20264, 20278, 20279}

In [33]:
set(tmr_feed_data['농가명'].unique()) & set(weight_data['농가명'].unique())

{20264}

In [34]:
set(milk_cls_feed_data['농가명'].unique()) & set(weight_data['농가명'].unique())

{20264}

In [35]:
ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
ict_cls_feed_data = ict_cls_feed_data[ict_cls_feed_data['개체번호'] == cow_id]
ict_cls_feed_data.set_index('측정일시' , inplace=True)
ict_cls_feed_data = ict_cls_feed_data.sort_values(by='측정일시')
ict_cls_feed_data

<ipython-input-35-580d9ea9dfac>:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')


,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,섭취량(kg),설정량(kg),급이시작일자,급이종료일자
측정일시,,,,,,,,,,,,
2020-01-20 00:00,345930,110262,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-01-20 00:00,2020-01-20 00:00
2020-01-21 00:00,345929,110261,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,5.0,0.0,2020-01-21 00:00,2020-01-21 00:00
2020-01-22 00:00,345928,110260,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-22 00:00,2020-01-22 00:00
2020-01-23 00:00,345927,110259,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-23 00:00,2020-01-23 00:00
2020-01-24 00:00,345926,110258,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-24 00:00,2020-01-24 00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-07 00:00,345699,110031,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-07 00:00,2020-09-07 00:00
2020-09-08 00:00,345698,110030,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-08 00:00,2020-09-08 00:00
2020-09-09 00:00,345697,110029,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-09 00:00,2020-09-09 00:00


In [36]:
# 데이터가 하루 단위로 기록이 되어 있는 지 확인
ict_cls_feed_data['급이시작일자'] = pd.to_datetime(ict_cls_feed_data['급이시작일자'])
ict_cls_feed_data['급이종료일자'] = pd.to_datetime(ict_cls_feed_data['급이종료일자'])
ict_cls_feed_data['급이차이일자'] = ict_cls_feed_data['급이종료일자'] - ict_cls_feed_data['급이시작일자']
ict_cls_feed_data['급이차이일자'].value_counts()


0 days    236
Name: 급이차이일자, dtype: int64

In [37]:
# 제대로 merge 되었는지 확인
merge_df = pd.merge(merge_df, ict_cls_feed_data[['섭취량(kg)']], how='outer' , left_index=True, right_index=True)
merge_df

,측정일시,일일 착유 횟수,착유량(L),임신기간,CO2 측정값,섭취량(kg)
2009-01-01 00:00:00,0.0,NaN,NaN,0.0,NaN,NaN
2009-01-02 00:00:00,0.0,NaN,NaN,0.0,NaN,NaN
2009-01-03 00:00:00,0.0,NaN,NaN,0.0,NaN,NaN
2009-01-04 00:00:00,0.0,NaN,NaN,0.0,NaN,NaN
2009-01-05 00:00:00,0.0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...
2020-12-14,NaN,NaN,NaN,NaN,570.0,NaN
2020-12-15,NaN,NaN,NaN,NaN,560.0,NaN
2020-12-16,NaN,NaN,NaN,NaN,560.0,NaN
2021-01-29,NaN,NaN,NaN,NaN,571.0,NaN


In [38]:
ict_cls_feed_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,섭취량(kg),설정량(kg),급이시작일자,급이종료일자,급이차이일자
측정일시,,,,,,,,,,,,,
2020-01-20 00:00,345930,110262,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-01-20,2020-01-20,0 days
2020-01-21 00:00,345929,110261,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,5.0,0.0,2020-01-21,2020-01-21,0 days
2020-01-22 00:00,345928,110260,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-22,2020-01-22,0 days
2020-01-23 00:00,345927,110259,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-23,2020-01-23,0 days
2020-01-24 00:00,345926,110258,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,6.0,0.0,2020-01-24,2020-01-24,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-07 00:00,345699,110031,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-07,2020-09-07,0 days
2020-09-08 00:00,345698,110030,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-08,2020-09-08,0 days
2020-09-09 00:00,345697,110029,NaN,100~200,20278,20120112010022,(주)애그리로보텍,1,3.0,0.0,2020-09-09,2020-09-09,0 days


In [39]:
merge_df[merge_df['섭취량(kg)'].notnull()]

,측정일시,일일 착유 횟수,착유량(L),임신기간,CO2 측정값,섭취량(kg)
2020-01-20 00:00,NaN,NaN,NaN,NaN,NaN,3.0
2020-01-21 00:00,NaN,NaN,NaN,NaN,NaN,5.0
2020-01-22 00:00,NaN,NaN,NaN,NaN,NaN,6.0
2020-01-23 00:00,NaN,NaN,NaN,NaN,NaN,6.0
2020-01-24 00:00,NaN,NaN,NaN,NaN,NaN,6.0
...,...,...,...,...,...,...
2020-09-07 00:00,NaN,NaN,NaN,NaN,NaN,3.0
2020-09-08 00:00,NaN,NaN,NaN,NaN,NaN,3.0
2020-09-09 00:00,NaN,NaN,NaN,NaN,NaN,3.0
2020-09-10 00:00,NaN,NaN,NaN,NaN,NaN,3.0


# get individual data

위에서 코드를 확인 했을 때 문제가 없는 것을 확인했으므로 이를 함수로 이용해서 각 개체별로 csv로 저장

In [40]:
cow_id = weight_data['개체번호'].unique()[0]
cow_id

20120112010022

In [41]:
def get_weight(data , cow_id):
    data = data[data['개체번호'] == cow_id]
    data = data.sort_values(by='측정일시')
    data['측정일시'] = pd.to_datetime(data['측정일시'])
    data.rename(columns={'측정일시' : 'time'} ,  inplace=True)
    return data[['time' , '체중(kg)']]

In [42]:
cow_df[cow_df['측정일시'] > 0]

,측정일시
2020-01-21,1
2020-01-22,1
2020-01-23,1
2020-01-24,1
2020-01-25,1
...,...
2020-09-07,1
2020-09-08,1
2020-09-09,1
2020-09-10,1


In [43]:
weight_id_data = get_weight(weight_data , cow_id)
weight_id_data

,time,체중(kg)
234,2020-01-21,639.0
233,2020-01-22,643.0
232,2020-01-23,646.0
231,2020-01-24,653.0
230,2020-01-25,655.0
...,...,...
4,2020-09-07,750.0
3,2020-09-08,750.0
2,2020-09-09,749.0
1,2020-09-10,751.0


In [44]:
def get_milk(data , cow_id):
    data = data[data['개체번호'] == cow_id]
    data = data[['개체번호' , '착유량(L)' , '측정일시']]
    dt = pd.date_range(start=start_date, end=end_date, freq='D')
    cow_df = pd.DataFrame(index = dt , columns = ['측정일시'])
    
    data['측정일시'] = pd.to_datetime(data['측정일시'])
    data['측정일시'] = data['측정일시'].dt.floor('H')  # 'H' stands for hour
    data = data.sort_values(by='측정일시')
    
    daily_counts = data.groupby([data['측정일시'].dt.date, '개체번호']).size().reset_index(name='일일 착유 횟수')
    daily_milk = data.groupby([data['측정일시'].dt.date, '개체번호'])['착유량(L)'].sum().reset_index()

    milk_agg_df = pd.merge(daily_counts, daily_milk, on=['측정일시', '개체번호'])
    milk_agg_df.reset_index(inplace=True)
    milk_agg_df.rename(columns={'측정일시': 'time'}, inplace=True)
    milk_agg_df['time'] = pd.to_datetime(milk_agg_df['time'])
    return milk_agg_df

    

In [45]:
milk_id_data = get_milk(milk_data , cow_id)
milk_id_data[milk_id_data['일일 착유 횟수'].notnull()]

,index,time,개체번호,일일 착유 횟수,착유량(L)
0,0,2020-01-20,20120112010022,2,48
1,1,2020-01-21,20120112010022,2,56
2,2,2020-01-22,20120112010022,2,74
3,3,2020-01-23,20120112010022,2,86
4,4,2020-01-24,20120112010022,2,86
...,...,...,...,...,...
231,231,2020-09-07,20120112010022,1,20
232,232,2020-09-08,20120112010022,1,22
233,233,2020-09-09,20120112010022,1,22
234,234,2020-09-10,20120112010022,1,21


In [46]:
milk_agg_df

,개체번호,일일 착유 횟수,착유량(L)
측정일시,,,
2020-01-20,20120112010022,2,48
2020-01-21,20120112010022,2,56
2020-01-22,20120112010022,2,74
2020-01-23,20120112010022,2,86
2020-01-24,20120112010022,2,86
...,...,...,...
2020-09-07,20120112010022,1,20
2020-09-08,20120112010022,1,22
2020-09-09,20120112010022,1,22


In [47]:
def get_breeding(data , cow_id):
    data = data[data['개체번호'] == cow_id]
    data = data[['개체번호' , '산차' , '수정일' , '수정차수' , '분만일']]
    data = data[data['개체번호'] == cow_id]
    data = data[data['분만일'].notnull()]
    data['수정일'] = pd.to_datetime(data['수정일'])
    data['분만일'] = pd.to_datetime(data['분만일'])
    return data

    
'''
# start , end 기간의 형식이므로 date_range를 이용해서 time stamp을 이용해서 기간을 구하고 기간을 넣음 
for index, row in breeding_data.iterrows():
    start, end = row['수정일'], row['분만일']
    print(start , end)
    pregnancy_dt = pd.date_range(start, end , freq = 'D')
    merge_df.loc[pregnancy_dt, '임신기간'] = 1

merge_df['임신기간'].fillna(0, inplace=True)    
'''

"\n# start , end 기간의 형식이므로 date_range를 이용해서 time stamp을 이용해서 기간을 구하고 기간을 넣음 \nfor index, row in breeding_data.iterrows():\n    start, end = row['수정일'], row['분만일']\n    print(start , end)\n    pregnancy_dt = pd.date_range(start, end , freq = 'D')\n    merge_df.loc[pregnancy_dt, '임신기간'] = 1\n\nmerge_df['임신기간'].fillna(0, inplace=True)    \n"

In [48]:
def get_env(data , farm_id):
    data = data[data['농가명'] == farm_id]
    data = data.sort_values(by='측정일시')
    data['측정일시'] = pd.to_datetime(data['측정일시'])
    data['측정일시'] = data['측정일시'].dt.floor('D') 
    data.rename(columns={'측정값(℃ · % · ppm)': '측정값'}, inplace=True)
    

    result_df = pd.DataFrame()
    result_df['측정일시'] = 0
    for equ in ['CO2' , '온도' , '습도']:
        data[data['장비종류'] == equ]
        equ_data = data[data['장비종류'] == equ][['측정일시' , '측정값']]
        equ_data.set_index('측정일시' , inplace = True)
        equ_data.rename(columns={'측정값': f'{equ} 측정값'}, inplace=True)
        equ_data = equ_data.sort_values(by='측정일시')
        result_df = result_df.merge(equ_data, on='측정일시', how='outer')
    result_df.rename(columns={'측정일시': "time"}, inplace=True)
    result_df['time'] = pd.to_datetime(result_df['time'])
    return result_df

In [49]:
env1_data[env1_data['농가명'] == farm_id]

,No,지역,규모,농가명,장비종류,제조사아이디,장비번호,측정일시,측정값
8403,8404,4.100000e+09,200~300,20278,온도,KILSOFT,1,2019-12-06,28.00
7970,7971,4.100000e+09,200~300,20278,CO2,KILSOFT,1,2020-07-27,610.00
8185,8186,4.100000e+09,200~300,20278,습도,KILSOFT,1,2020-07-27,73.00
8402,8403,4.100000e+09,200~300,20278,온도,KILSOFT,1,2020-07-27,29.00
8184,8185,4.100000e+09,200~300,20278,습도,KILSOFT,1,2020-07-28,80.00
...,...,...,...,...,...,...,...,...,...
8188,8189,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-29,7.75
8187,8188,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-30,5.92
7972,7973,4.100000e+09,200~300,20278,습도,thekpmcorp,1,2022-12-30,73.00
8186,8187,4.100000e+09,200~300,20278,온도,thekpmcorp,1,2022-12-31,75.96


In [50]:
env_id_data = get_env(env1_data , farm_id)
env_id_data[env_id_data['CO2 측정값'].notnull()]

,time,CO2 측정값,온도 측정값,습도 측정값
0,2020-07-27,610.0,29.0,73.0
1,2020-07-28,607.0,26.0,80.0
2,2020-07-29,601.0,27.0,82.0
3,2020-07-30,603.0,28.0,82.0
4,2020-07-31,610.0,29.0,79.0
...,...,...,...,...
88,2020-12-14,570.0,-2.0,63.0
89,2020-12-15,560.0,-4.0,82.0
90,2020-12-16,560.0,-10.0,82.0
91,2021-01-29,571.0,-2.0,82.0


In [51]:
CO2_data = env1_data[env1_data['장비종류'] == 'CO2'][['측정일시' , '측정값']]
CO2_data.set_index('측정일시' , inplace = True)
CO2_data

,측정값
측정일시,
2020-07-27,610.0
2020-07-28,607.0
2020-07-29,601.0
2020-07-30,603.0
2020-07-31,610.0
...,...
2020-12-14,570.0
2020-12-15,560.0
2020-12-16,560.0


In [52]:
def get_feed(data , cow_id):
    data = data[data['개체번호'] == cow_id]
    data['측정일시'] = pd.to_datetime(data['측정일시'])
    data.rename(columns={'측정일시' : "time"} , inplace=True)
    return data[['time' , '섭취량(kg)']]

    

In [53]:
ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
feed_data = get_feed(ict_cls_feed_data , cow_id)
feed_data

<ipython-input-53-11dfd6fff53b>:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)


,time,섭취량(kg)
345695,2020-09-11,3.0
345696,2020-09-10,3.0
345697,2020-09-09,3.0
345698,2020-09-08,3.0
345699,2020-09-07,3.0
...,...,...
345926,2020-01-24,6.0
345927,2020-01-23,6.0
345928,2020-01-22,6.0
345929,2020-01-21,5.0


In [54]:
feed_data

,time,섭취량(kg)
345695,2020-09-11,3.0
345696,2020-09-10,3.0
345697,2020-09-09,3.0
345698,2020-09-08,3.0
345699,2020-09-07,3.0
...,...,...
345926,2020-01-24,6.0
345927,2020-01-23,6.0
345928,2020-01-22,6.0
345929,2020-01-21,5.0


In [55]:
unique_combinations = weight_data[['농가명', '개체번호']].drop_duplicates()
unique_combinations


,농가명,개체번호
0,20278,20120112010022
235,20278,20120423010010
236,20278,20130126010057
444,20278,20130528010023
487,20278,20130609010024
...,...,...
59055,21133,20200128010113
59148,21133,20200129010114
59186,21133,20200130010115
59267,21133,20200202010116


In [56]:
weight_data = pd.read_excel('./raw_data/낙농_ICT데이터_체중정보.xlsx', header=1)
milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')

breeding_data = pd.read_excel('./raw_data/낙농_개체별_번식이력.xlsx' , header=1)

env1_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경관리기정보.xlsx' , header=1)
env2_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경제어기정보.xlsx' , header=1)

gen_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_일반급이기정보.xlsx' , header=1)
ict_cls_feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
tmr_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_TMR급이기정보.xlsx' , header=1)
milk_cls_feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_착유군사급이기정보.xlsx' , header=1)


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-56-3211b4781551>:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no defau

In [62]:
import numpy as np
for iter_idx , (idx , row) in enumerate(unique_combinations.iterrows()):
    farm_id  , cow_id = row['농가명'] , row['개체번호']
    #print(farm_id , cow_id)

    weight_id_data = get_weight(weight_data , cow_id)
    milk_id_data = get_milk(milk_data , cow_id)
    feed_id_data = get_feed(ict_cls_feed_data , cow_id)
    env_id_data = get_env(env1_data , farm_id)

    print(weight_id_data.shape , milk_id_data.shape ,feed_id_data.shape , breed_id_data.shape)
    merge_df = pd.merge(weight_id_data , milk_id_data  , how='outer', on='time')
    merge_df = pd.merge(merge_df , feed_id_data  , how='outer',on='time')
    merge_df = pd.merge(merge_df , env_id_data  , how='outer',on='time')
    merge_df.to_csv(f'./merge_data/{farm_id}_{cow_id}_merge_data.csv')
    
    print(f'{cow_id} is saved')
    

<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

(235, 2) (236, 5) (236, 2) (6, 5)
20120112010022 is saved
(1, 2) (2, 5) (2, 2) (6, 5)
20120423010010 is saved
(208, 2) (289, 5) (211, 2) (6, 5)
20130126010057 is saved
(43, 2) (44, 5) (44, 2) (6, 5)
20130528010023 is saved


<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

(218, 2) (305, 5) (220, 2) (6, 5)
20130609010024 is saved
(279, 2) (565, 5) (280, 2) (6, 5)
20130816010079 is saved
(219, 2) (231, 5) (221, 2) (6, 5)
20140113010011 is saved
(249, 2) (397, 5) (250, 2) (6, 5)
20140417010025 is saved
(280, 2) (390, 5) (281, 2) (6, 5)
20140918010026 is saved
(263, 2) (307, 5) (264, 2) (6, 5)
20140925010027 is saved
(258, 2) (260, 5) (260, 2) (6, 5)
20140930010028 is saved
(242, 2) (479, 5) (243, 2) (6, 5)
20140930010029 is saved
(244, 2) (418, 5) (245, 2) (6, 5)
20141014010030 is saved


<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

...

(233, 2) (574, 5) (458, 2) (6, 5)
20180427020022 is saved
(189, 2) (546, 5) (485, 2) (6, 5)
20180509020119 is saved
(170, 2) (171, 5) (0, 2) (6, 5)
20180520020099 is saved
(120, 2) (478, 5) (457, 2) (6, 5)
20180520020111 is saved
(112, 2) (463, 5) (450, 2) (6, 5)


<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

20180614020078 is saved
(93, 2) (445, 5) (491, 2) (6, 5)
20180723020081 is saved
(125, 2) (466, 5) (464, 2) (6, 5)
20180802020063 is saved
(12, 2) (359, 5) (487, 2) (6, 5)
20180804020046 is saved
(86, 2) (453, 5) (472, 2) (6, 5)
20180821020070 is saved
(4, 2) (346, 5) (471, 2) (6, 5)
20180823020006 is saved


<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['측정일시'] = pd.to_datetime(data['측정일시'])
<ipython-input-52-ea6a444f1497>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'측정일시' : "time"} , inplace=True)
<ipython-input-52-ea6a444f1497>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

ValueError: You are trying to merge on int64 and datetime64[ns] columns. If you wish to proceed you should use pd.concat